In [1]:
!pip install pynput
!pip install  torch_lr_finder
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 6.5 MB/s eta 0:00:00
  Created wheel for evdev: filename=evdev-1.6.1-cp310-cp310-linux_x86_64.whl size=45847 sha256=2ab302b689bac2313d25cc8e0459bfcda8b3c7a4173974258fd2d43e517eaac3
  Stored in directory: /root/.cache/pip/wheels/bb/6e/51/0c0f5677bd58c532c42e13e7d93803fbf8d5d3bf5bfc5b0220
Successfully built evdev


In [2]:
!mkdir store

mkdir: cannot create directory ‘store’: File exists


In [3]:
!ls store

'epoch=25-step=13468.ckpt'  'epoch=28-step=15022.ckpt'
'epoch=26-step=13986.ckpt'  'epoch=36-step=19166.ckpt'
'epoch=27-step=14504.ckpt'  'epoch=4-step=25.ckpt'


In [5]:
!rm -r /kaggle/working/yolov3_object_detection/
%cd /kaggle/working
!git clone https://github.com/venkyyuvy/yolov3_object_detection.git
%cd /kaggle/working/yolov3_object_detection
!git checkout modified_callback
!python main.py

/kaggle/working
Cloning into 'yolov3_object_detection'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 138 (delta 87), reused 96 (delta 48), pack-reused 0
Receiving objects: 100% (138/138), 35.82 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/kaggle/working/yolov3_object_detection
Branch 'modified_callback' set up to track remote branch 'modified_callback' from 'origin'.
Switched to a new branch 'modified_callback'
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/co

In [11]:
from model import YOLOv3
import torch
fname = '../store/epoch=38-step=20202.ckpt'
checkpoint = torch.load(fname, map_location=torch.device('cpu'))
model_state_dict = checkpoint['state_dict']
model = YOLOv3(num_classes=20)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [23]:
import config
from dataset import YOLODataModule

yolo_datamodule = YOLODataModule(
        train_csv_path=config.DATASET + "/train.csv",
        test_csv_path=config.DATASET + "/test.csv",
    )

yolo_datamodule.setup()

In [27]:
config.DEVICE


'cuda'

In [ ]:
from utils import get_evaluation_bboxes
model.eval()
model = model.to(config.DEVICE)
pred_boxes, true_boxes = get_evaluation_bboxes(
        loader=yolo_datamodule.test_dataloader(),
        model=model,
        iou_threshold=config.NMS_IOU_THRESH,
        anchors=config.ANCHORS,
        threshold=config.CONF_THRESHOLD,
    )

           